In [11]:
# ============================================================
# Statistical Business Analysis Project - CORRECTED VERSION
# ============================================================

# -------------------------------
# STEP 1: Import Required Libraries
# -------------------------------
import pandas as pd
import numpy as np
import matplotlib
matplotlib.use('Agg')  # Non-interactive backend to avoid plot errors
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import warnings
warnings.filterwarnings('ignore')

# -------------------------------
# STEP 2: Load Given Data Files (with error handling)
# -------------------------------
try:
    churn_df = pd.read_csv(r"C:\Users\Dell\Downloads\customer_churn (1).csv")
    print("Customer Churn Data Loaded Successfully")
    print(churn_df.head())
    print(f"Churn data shape: {churn_df.shape}")
except FileNotFoundError:
    churn_df = None
    print("Customer Churn file not found - skipping churn analysis")

try:
    sales_df = pd.read_csv(r"C:\Users\Dell\Downloads\sales_data (5).csv")
    print("\nSales Data Loaded Successfully")
    print(sales_df.head())
    print(f"Sales data shape: {sales_df.shape}")
    print("Sales columns:", sales_df.columns.tolist())
except FileNotFoundError:
    sales_df = None
    print("Sales data file not found - analysis will skip sales sections")

# Proceed only if sales_df is loaded
if sales_df is None:
    print("ERROR: Sales data required for analysis. Please provide sales_data (5).csv")
    exit()

# -------------------------------
# DAY 1: DESCRIPTIVE STATISTICS
# -------------------------------
print("\n--- DESCRIPTIVE STATISTICS (Sales Data) ---")
print(sales_df.describe(include='all'))  # include='all' for better overview

print("\nMedian Values:")
print(sales_df.select_dtypes(include=[np.number]).median())  # Only numeric, avoids FutureWarning

print("\nMode Values:")
mode_df = sales_df.mode()
if not mode_df.empty:
    print(mode_df.iloc[0])
else:
    print("No mode computed (all unique values)")

# -------------------------------
# DAY 2: DATA DISTRIBUTION ANALYSIS
# -------------------------------
if 'Sales' in sales_df.columns:
    plt.figure(figsize=(8, 5))
    sns.histplot(data=sales_df, x="Sales", kde=True)
    plt.title("Sales Distribution")
    plt.xlabel("Sales")
    plt.ylabel("Frequency")
    plt.savefig("sales_distribution.png")  # Save instead of show()
    plt.close()  # Close to free memory
    print("Sales distribution plot saved as 'sales_distribution.png'")

    # Normality Test (sample if large dataset)
    sales_sample = sales_df["Sales"].dropna()
    if len(sales_sample) <= 5000:
        stat, p_value = stats.shapiro(sales_sample)
    else:
        stat, p_value = stats.kstest(sales_sample, 'norm', args=(sales_sample.mean(), sales_sample.std()))
    print("\nNormality Test p-value:", p_value)

    if p_value > 0.05:
        print("Sales data appears normally distributed (fail to reject null)")
    else:
        print("Sales data is NOT normally distributed (reject null)")
else:
    print("Warning: 'Sales' column not found")

# -------------------------------
# DAY 3: CORRELATION ANALYSIS
# -------------------------------
numeric_cols = ['Sales', 'Marketing_Spend', 'Revenue']
available_cols = [col for col in numeric_cols if col in sales_df.columns]
if len(available_cols) >= 2:
    corr_matrix = sales_df[available_cols].corr(numeric_only=True)
    print("\nCorrelation Matrix:")
    print(corr_matrix)

    plt.figure(figsize=(8, 6))
    sns.heatmap(corr_matrix, annot=True, cmap="coolwarm", center=0)
    plt.title("Correlation Heatmap")
    plt.tight_layout()
    plt.savefig("correlation_heatmap.png")
    plt.close()
    print("Correlation heatmap saved as 'correlation_heatmap.png'")
else:
    print("Insufficient numeric columns for correlation")

# -------------------------------
# DAY 4: HYPOTHESIS TESTING
# -------------------------------
results = []

if 'Sales' in sales_df.columns:
    # Test 1: One-sample t-test (skip if NaNs)
    sales_clean = sales_df["Sales"].dropna()
    if len(sales_clean) > 0:
        t1_stat, t1_p = stats.ttest_1samp(sales_clean, 50000)
        results.append(f"One-sample t-test p-value: {t1_p:.4f}")

    # Test 2: Independent t-test (North vs South) - check if regions exist
    if 'Region' in sales_df.columns:
        north_sales = sales_df[sales_df["Region"] == "North"]["Sales"].dropna()
        south_sales = sales_df[sales_df["Region"] == "South"]["Sales"].dropna()
        if len(north_sales) > 1 and len(south_sales) > 1:
            t2_stat, t2_p = stats.ttest_ind(north_sales, south_sales, equal_var=False)  # Welch's t-test safer
            results.append(f"Independent t-test p-value (North vs South): {t2_p:.4f}")

        # Test 3: ANOVA (All regions)
        groups = [group["Sales"].dropna() for _, group in sales_df.groupby("Region") if len(group["Sales"].dropna()) > 1]
        if len(groups) > 1:
            anova_stat, anova_p = stats.f_oneway(*groups)
            results.append(f"ANOVA p-value (regions): {anova_p:.4f}")

print("\n--- HYPOTHESIS TEST RESULTS ---")
for r in results:
    print(r)

# Save hypothesis results
with open("hypothesis_tests_results.txt", "w") as f:
    for r in results:
        f.write(r + "\n")

# -------------------------------
# DAY 5: CONFIDENCE INTERVALS
# -------------------------------
if 'Revenue' in sales_df.columns:
    revenue_clean = sales_df["Revenue"].dropna()
    if len(revenue_clean) > 0:
        mean_rev = revenue_clean.mean()
        std_rev = revenue_clean.std()
        n = len(revenue_clean)
        z = stats.norm.ppf(0.975)
        margin_error = z * (std_rev / np.sqrt(n))
        ci_lower = mean_rev - margin_error
        ci_upper = mean_rev + margin_error
        print(f"\n95% Confidence Interval for Revenue (n={n}):")
        print(f"Lower Bound: {ci_lower:.2f}")
        print(f"Upper Bound: {ci_upper:.2f}")
else:
    print("\n'Revenue' column not found for CI calculation")

# -------------------------------
# DAY 6: REGRESSION ANALYSIS
# -------------------------------
if all(col in sales_df.columns for col in ['Marketing_Spend', 'Revenue']):
    X = sales_df[["Marketing_Spend"]].dropna()
    y = sales_df.loc[X.index, "Revenue"]  # Align indices

    X = sm.add_constant(X)
    model = sm.OLS(y, X).fit()

    print("\n--- REGRESSION SUMMARY ---")
    print(model.summary())
    print(f"\nR-squared Value: {model.rsquared:.4f}")
else:
    print("\nMissing columns for regression (need Marketing_Spend and Revenue)")

# -------------------------------
# DAY 7: BUSINESS INSIGHTS (using available corr_matrix)
# -------------------------------
print("\n--- BUSINESS INSIGHTS & RECOMMENDATIONS ---")

corr_matrix_safe = globals().get('corr_matrix', pd.DataFrame())
if 'Marketing_Spend' in corr_matrix_safe.index and 'Revenue' in corr_matrix_safe.columns:
    corr_val = corr_matrix_safe.loc["Marketing_Spend", "Revenue"]
    if abs(corr_val) > 0.6:
        print("✔ Strong relationship between marketing spend and revenue (|r| > 0.6)")

if results:  # Use saved p-values
    for r in results:
        if 't1' in r.lower() and float(r.split(':')[1]) <= 0.05:
            print("✔ Average sales significantly differ from 50,000 target")
        if 'anova' in r.lower() and float(r.split(':')[1]) <= 0.05:
            print("✔ Sales significantly vary across regions")

print("\nRecommendations:")
print("- Increase marketing spend in high-performing regions")
print("- Use regression model for revenue forecasting")
print("- Monitor regional differences via ANOVA")
print("- Apply A/B testing for campaigns")

print("\nPROJECT COMPLETED SUCCESSFULLY")
print("Generated files: hypothesis_tests_results.txt, sales_distribution.png, correlation_heatmap.png")


Customer Churn Data Loaded Successfully
  CustomerID  Tenure  MonthlyCharges  TotalCharges        Contract  \
0     C00001       6              64          1540        One year   
1     C00002      21             113          1753  Month-to-month   
2     C00003      27              31          1455        Two year   
3     C00004      53              29          7150  Month-to-month   
4     C00005      16             185          1023        One year   

      PaymentMethod PaperlessBilling  SeniorCitizen  Churn  
0       Credit Card               No              1      0  
1  Electronic Check              Yes              1      0  
2       Credit Card               No              1      0  
3  Electronic Check               No              1      0  
4  Electronic Check               No              1      0  
Churn data shape: (500, 9)

Sales Data Loaded Successfully
         Date     Product  Quantity  Price Customer_ID Region  Total_Sales
0  2024-01-01       Phone         7  37